`secretflow.utils.simulation.data.dataframe.create_df` 是一个用于创建联邦数据框的函数，支持根据数据源和数据分区方案生成 `HDataFrame` 或 `VDataFrame`。它允许将数据按行或按列分割，并支持多种数据分区方法。

### 函数签名

```
python复制代码create_df(
    source: str | DataFrame | Callable,
    parts: List[PYU] | Dict[PYU, float | tuple],
    axis: int = 0,
    shuffle: bool = False,
    random_state: int | None = None,
    aggregator: Aggregator | None = None,
    comparator: Comparator | None = None,
    split_method: SPLIT_METHOD = SPLIT_METHOD.IID,
    label_column: str | None = None,
    **kwargs
) → HDataFrame | VDataFrame
```

### 参数

- **source**: 数据源，可以是文件路径、`pandas.DataFrame` 对象，或一个返回 `pandas.DataFrame` 的 callable 对象。

- **parts**: 数据分区方案。

  - 如果是 `List[PYU]` 类型，则数据将尽可能均匀地分配给每个 PYU。

  - 如果是 

    ```
    Dict[PYU, float | tuple]
    ```

     类型，PYU 的值决定了数据分配的比例或范围。具体可以是：

    - `float`：每个 PYU 分配的数据比例。
    - `tuple`：一个区间（左闭右开），用于指定数据分配的范围。

- **axis**: 可选，分割的轴。

  - `0`：按行分割，返回水平分区的联邦数据框（`HDataFrame`）。
  - `1`：按列分割，返回垂直分区的联邦数据框（`VDataFrame`）。

- **shuffle**: 可选，是否在分割数据之前对数据进行洗牌。

- **random_state**: 可选，用于洗牌的随机种子。

- **aggregator**: 可选，仅在 `axis=0` 时使用，指定用于水平分区的聚合器。

- **comparator**: 可选，仅在 `axis=0` 时使用，指定用于水平分区的比较器。

- **split_method**: 可选，指定数据分区的方法。默认为 `SPLIT_METHOD.IID`（独立同分布）。其他方法包括：

  - `SPLIT_METHOD.DIRICHLET`：Dirichlet 分区方法。
  - `SPLIT_METHOD.LABEL_SKEW`：标签偏斜分区方法。

- **label_column**: 可选，指定用于标签偏斜的标签列名。

- **kwargs**: 可选，其他分割方法的参数，如 `num_classes`（用于 Dirichlet 分区）或 `alpha`（用于 Dirichlet 分区）等。

### 返回值

- 如果 `axis=0`，则返回 `HDataFrame`（水平分区的联邦数据框）。
- 如果 `axis=1`，则返回 `VDataFrame`（垂直分区的联邦数据框）。

In [19]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob'], address='local')
alice = sf.PYU('alice')
bob = sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(parties=['alice', 'bob']))

The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-20 20:40:08,776	INFO worker.py:1724 -- Started a local Ray instance.


In [20]:
import pandas as pd
from secretflow.utils.simulation.data.dataframe import create_df
from secretflow.utils.simulation.data import SPLIT_METHOD

# 示例数据
df = pd.DataFrame({'f1': [1, 2, 3, 4], 'f3': [11, 12, 13, 14]})

In [21]:
# 创建水平分区的 HDataFrame，数据均匀分配
hdf = create_df(df, [alice, bob], axis=0)

# 创建垂直分区的 VDataFrame，指定每个 PYU 的数据比例
vdf = create_df(df, {alice: 0.3, bob: 0.7}, axis=1)

# 创建水平分区的 HDataFrame，指定每个 PYU 的数据范围
hdf = create_df(df, {alice: (0, 1), bob: (1, 4)})

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


In [22]:
# 使用 Dirichlet 分区方法创建水平分区的 HDataFrame
hdf = create_df(df, [alice, bob], axis=0, split_method=SPLIT_METHOD.DIRICHLET, num_classes=2, alpha=1000)

KeyError: None

In [14]:
# 使用标签偏斜分区方法创建水平分区的 HDataFrame
hdf = create_df(df, [alice, bob], axis=0, split_method=SPLIT_METHOD.LABEL_SKEW, label_column='f1',skew_ratio=0.5)

AttributeError: LABEL_SKEW